In [1]:
import os

from d_2021.config.path import *
from d_2021.utils.import_utils import *
from d_2021.utils.math_utils import *
from d_2021.utils.common_utils import *
from tqdm import *
import pandas as pd

In [2]:
def data_pre_threat():
    if os.path.exists(md_df_path):
        md_df_3 = pd.read_csv(md_df_path)
        md_names = md_df_3.columns.tolist()
    else:
        md_names, md_df = get_objs_in_csv("../dataset/train/Molecular_Descriptor.csv")
        print('总特征数:', len(md_names) - 1)
        error_names = []
        # 使用箱线法去除异常值出现大于100次的列
        for md_name in md_names[1:]:
            row = md_df[md_name].tolist()
            # row = str_list_2_int(row)
            min_num, Q1, Q2, Q3, max_num = get_box_plot(row, md_name, False)
            # 计算异常值数量
            error_count = 0
            for i in range(len(row)):
                cell = md_df[md_name][i]
                if cell < min_num:
                    error_count += 1
                    md_df.loc[i, md_name] = min_num
                if cell > max_num:
                    error_count += 1
                    md_df.loc[i, md_name] = max_num
            if error_count > 100:
                error_names.append(md_name)
        print('异常值数量:', len(error_names))
        md_df_2 = remove_names(md_df, md_names, error_names)
        # 含0大于0.9的列
        zero_names = []
        # 计算列内含0大于90%的列
        for md_name in md_names[1:]:
            row = md_df_2[md_name]
            p = count_zero(row)
            if p > 0.9:
                zero_names.append(md_name)
        print('0值大于90%数量:', len(zero_names))
        md_df_3 = remove_names(md_df, md_names, zero_names)
        print('筛选后总数', len(md_names) - 1)
        md_df_3.to_csv(md_df_path, index=False)
    return md_df_3, md_names

In [3]:
pre_threat_df, pre_threat_names = data_pre_threat()
pre_threat_df

,SMILES,nAcid,ALogP,ALogp2,AMR,apol,naAromAtom,nAromBond,nAtom,nHeavyAtom,...,MW,WTPT-1,WTPT-2,WTPT-3,WTPT-4,WTPT-5,WPATH,WPOL,XLogP,Zagreb
0,Oc1ccc2O[C@H]([C@H](Sc2c1)C3CCCC3)c4ccc(OCCN5C...,0.0,-0.2860,0.081796,126.1188,74.170169,12.0,12.0,64.0,31.0,...,439.218115,64.771680,2.089409,15.471445,8.858910,3.406628,3011.0,47.0,4.666,166.0
1,Oc1ccc2O[C@H]([C@H](Sc2c1)C3CCCCCC3)c4ccc(OCCN...,0.0,-0.8620,0.743044,131.9420,80.357341,12.0,12.0,70.0,33.0,...,467.249415,68.960024,2.089698,15.486947,8.863774,3.406648,3516.0,54.0,5.804,174.0
2,Oc1ccc(cc1)[C@H]2Sc3cc(O)ccc3O[C@H]2c4ccc(OCCN...,0.0,0.7296,0.532316,139.9304,74.064997,18.0,18.0,62.0,33.0,...,463.181729,68.748923,2.083301,18.011114,11.390412,3.406644,3542.0,52.0,2.964,176.0
3,Oc1ccc2O[C@H]([C@@H](CC3CCCCC3)Sc2c1)c4ccc(OCC...,0.0,-0.3184,0.101379,133.4822,80.357341,12.0,12.0,70.0,33.0,...,467.249415,68.883696,2.087385,15.468365,8.857943,3.406624,3594.0,50.0,6.015,174.0
4,Oc1ccc2O[C@H]([C@@H](Cc3ccccc3)Sc2c1)c4ccc(OCC...,0.0,1.3551,1.836296,143.1903,76.356583,18.0,18.0,64.0,33.0,...,461.202465,68.883696,2.087385,15.468365,8.857943,3.406624,3594.0,50.0,4.462,174.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1969,COc1cc(OC)cc(\C=C\c2cccc(OS(=O)(=O)C3CC4OC3C(=...,0.0,1.8193,3.309852,177.6817,89.159790,24.0,24.0,73.0,43.0,...,598.166139,88.709996,2.063023,25.470481,20.590936,0.000000,7121.0,70.0,2.526,236.0
1970,Oc1ccc(cc1)C2=C(C3OC2CC3S(=O)(=O)Oc4cccc(\C=C\...,0.0,1.6903,2.857114,167.6057,82.972618,24.0,24.0,67.0,41.0,...,570.134839,84.662088,2.064929,24.928962,20.590936,0.000000,6171.0,66.0,1.884,228.0
1971,Oc1ccc(cc1)C2=C(C3OC2CC3S(=O)(=O)Oc4ccc(\C=C\c...,0.0,1.6903,2.857114,167.6057,82.972618,24.0,24.0,67.0,41.0,...,570.134839,84.660642,2.064894,24.923083,20.590936,0.000000,6421.0,66.0,1.884,228.0
1972,Oc1ccc(cc1)C2=C([C@H]3O[C@H]2C[C@@H]3S(=O)(=O)...,0.0,1.3365,1.786232,125.5605,63.287860,18.0,18.0,51.0,31.0,...,436.098059,64.171346,2.070043,19.841924,16.326873,0.000000,2583.0,50.0,0.782,174.0


In [4]:
def get_sequence(m_df, names, csv_path):
    """
    得到计算好的序列
    :param m_df: 数据表
    :param names: 名称
    :param csv_path: 路径
    :return: 处理好的序列
    """
    if os.path.exists(csv_path):
        sequence = pd.read_csv(csv_path)
    else:
        sequence = get_sequence_divide_mean(m_df, names)
        sequence.to_csv(csv_path, index=False)
    return sequence

In [5]:
def get_mean_sequence():
    era_names, era_df = get_objs_in_csv("../dataset/train/ERa_activity.csv")
    # 得到母序列均值商
    parent = get_sequence(era_df, ['pIC50'], csv1_path)
    # 得到子序列均值商
    child_names1 = pre_threat_names[1:]
    child = get_sequence(pre_threat_df, child_names1, csv2_path)
    return parent, child, child_names1

In [6]:
parent_sequence, child_sequence, child_names = get_mean_sequence()

  0%|          | 0/402 [00:00<?, ?it/s]C:\Project\My\mathematical-model\d_2021\utils\math_utils.py:58: RuntimeWarning: invalid value encountered in double_scalars
  t_df.loc[i, name] = t1_df[i] / t_df_mean
100%|██████████| 402/402 [00:46<00:00,  8.64it/s]


In [12]:
def cal_ab():
    local_min = 100
    local_max = 0
    for name in pre_threat_names[1:]:
        # 对每一个子序列，计算对应值
        sequence = child_sequence[name]
        for i in range(len(sequence)):
            # 每一行的数相减
            t = abs(sequence[i] - parent_sequence['pIC50'][i])
            if t < local_min:
                local_min = t
            if t > local_max:
                local_max = t
    return local_min, local_max

In [13]:
a, b = cal_ab()

In [10]:
def cal_relevancy():
    copy_child = child_sequence.copy()
    p = 0.5
    for name in tqdm(child_names):
        # 对每一个子序列，计算对应值
        sequence = copy_child[name]
        for i in range(len(sequence)):
            # 计算关联度
            sequence[i] = (a + p * b) / (abs(sequence[i] - parent_sequence['pIC50'][i]) + p * b)
    copy_child.to_csv(csv3_path, index=False)
    return copy_child

In [11]:
relevancy_df = cal_relevancy()

100%|██████████| 402/402 [05:26<00:00,  1.23it/s]


In [14]:
relevancy_df

,nAcid,ALogP,ALogp2,AMR,apol,naAromAtom,nAromBond,nAtom,nHeavyAtom,nH,...,MW,WTPT-1,WTPT-2,WTPT-3,WTPT-4,WTPT-5,WPATH,WPOL,XLogP,Zagreb
0,NaN,0.997617,0.998047,0.999660,0.999883,0.999183,0.999132,0.999944,0.999696,0.999746,...,0.999729,0.999723,0.999557,0.999539,0.999561,0.999156,0.999886,0.999560,0.999542,0.999692
1,NaN,0.996958,0.998540,0.999848,0.999849,0.999294,0.999243,0.999762,0.999917,0.999362,...,0.999951,0.999946,0.999668,0.999652,0.999673,0.999266,0.999687,0.999904,0.998831,0.999884
2,NaN,0.998999,0.998329,0.999864,0.999902,0.999797,0.999720,0.999905,0.999827,0.999998,...,0.999845,0.999851,0.999574,0.999814,0.999971,0.999177,0.999760,0.999748,0.999582,0.999815
3,NaN,0.997619,0.998104,0.999802,0.999915,0.999228,0.999176,0.999829,0.999850,0.999428,...,0.999884,0.999878,0.999600,0.999584,0.999605,0.999200,0.999704,0.999705,0.998786,0.999818
4,NaN,0.999927,0.999172,0.999995,0.999952,0.999885,0.999808,0.999946,0.999915,0.999773,...,0.999925,0.999943,0.999665,0.999649,0.999670,0.999265,0.999639,0.999770,0.999539,0.999883
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1969,NaN,0.998984,0.999509,0.999085,0.999161,0.999057,0.999160,0.999207,0.999072,0.999377,...,0.999070,0.999064,0.999888,0.998889,0.997797,0.998574,0.996969,0.999100,0.999905,0.999019
1970,NaN,0.999092,0.999706,0.999152,0.999253,0.998992,0.999094,0.999324,0.999116,0.999585,...,0.999115,0.999106,0.999821,0.998878,0.997732,0.998639,0.997497,0.999168,0.999631,0.999035
1971,NaN,0.999527,0.999859,0.999588,0.999688,0.999427,0.999529,0.999760,0.999551,0.999980,...,0.999550,0.999541,0.999744,0.999313,0.998166,0.998205,0.997774,0.999603,0.999196,0.999470
1972,NaN,0.999959,0.999200,0.999820,0.999773,0.999942,0.999865,0.999715,0.999863,0.999531,...,0.999883,0.999874,0.999709,0.999858,0.998955,0.998166,0.999784,0.999827,0.998578,0.999940


In [15]:
a,b

(6.07892792214848e-07, 1303.552405293812)